### Adjust Word Group In Youtube Sentence Word Usage Analysis

In [145]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [146]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [147]:
# language pair
lang_folder = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 200 # native word end index

# word all usage in sent
word_use_num_max = 7  # 5
word_use_num_min = 1  # 1

# youtube sentence usage for remain word 
youtube_sent_usage = "selected"  # "all","selected"

In [148]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Adjust Word Group In Youtube Sentence Word Usage Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [149]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Word Usage

In [150]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [151]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
317922,puisetil,5
317923,shouzheng,5
317924,steinam,5
317925,cochezle,5


In [152]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,de,16588988
1,1,je,16386475
2,2,pas,11547876
3,3,le,10592792
4,4,la,9939090
...,...,...,...
195,195,nos,332905
196,196,cela,330730
197,197,nom,329252
198,198,prendre,327639


In [153]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/\
2-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word6.xlsx")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,comment ça on vous veut du mal pas du tout on ...,261,266,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY,https://www.youtube.com/watch?v=DFejViks2uY&t=...
1,si si si si si si si si si si si si si si donc...,8213,8216,pouvons pas vous dire ce qui va se passer si n...,tlPvaDllxiI,https://www.youtube.com/watch?v=tlPvaDllxiI&t=...
2,chez nous fait tout questce quelle ne fait pas...,1333,1337,caroline chez nous fait tout questce quelle ne...,rfx823Fyd10,https://www.youtube.com/watch?v=rfx823Fyd10&t=...
3,cest très très bon cest très très très très bo...,1526,1533,cest très très bon cest très très très très bo...,T5I_cTc13Zw,https://www.youtube.com/watch?v=T5I_cTc13Zw&t=...
4,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...
...,...,...,...,...,...,...
128056,on la on va,150,151,on la on va essayer de décortiquer commencé à ...,k0CLXlk4z68,NaN
128057,et là ça se,14,15,et là ça se complique,60HvI3CcGFs,NaN
128058,ça ma ça ma,956,957,ça ma ça ma amené à me interrogé le déjà la pr...,Wn38oL03yM8,NaN
128059,de la de la,685,686,et leurs corps étaient rendus présent par la m...,Wn38oL03yM8,NaN


In [154]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,comment ça on vous veut du mal pas du tout on ...,261,266,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY,https://www.youtube.com/watch?v=DFejViks2uY&t=...,"[74, 14, 19, 6, 152, 30, 156, 2, 30, 33, 19, 6..."
1,si si si si si si si si si si si si si si donc...,8213,8216,pouvons pas vous dire ce qui va se passer si n...,tlPvaDllxiI,https://www.youtube.com/watch?v=tlPvaDllxiI&t=...,"[28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 2..."
2,chez nous fait tout questce quelle ne fait pas...,1333,1337,caroline chez nous fait tout questce quelle ne...,rfx823Fyd10,https://www.youtube.com/watch?v=rfx823Fyd10&t=...,"[159, 26, 40, 33, 85, 104, 11, 40, 2, 1, 50, 2..."
3,cest très très bon cest très très très très bo...,1526,1533,cest très très bon cest très très très très bo...,T5I_cTc13Zw,https://www.youtube.com/watch?v=T5I_cTc13Zw&t=...,"[10, 81, 81, 78, 10, 81, 81, 81, 81, 78, 17, 4..."
4,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...,"[65, 99, 47, 134, 131, 5, 182, 10, 40, 32, 24,..."
...,...,...,...,...,...,...,...
128056,on la on va,150,151,on la on va essayer de décortiquer commencé à ...,k0CLXlk4z68,NaN,"[19, 4, 19, 42]"
128057,et là ça se,14,15,et là ça se complique,60HvI3CcGFs,NaN,"[8, 61, 14, 48]"
128058,ça ma ça ma,956,957,ça ma ça ma amené à me interrogé le déjà la pr...,Wn38oL03yM8,NaN,"[14, 43, 14, 43]"
128059,de la de la,685,686,et leurs corps étaient rendus présent par la m...,Wn38oL03yM8,NaN,"[0, 4, 0, 4]"


In [155]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [156]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,comment ça on vous veut du mal pas du tout on ...,261,266,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY,https://www.youtube.com/watch?v=DFejViks2uY&t=...,"[74, 14, 19, 6, 152, 30, 156, 2, 30, 33, 19, 6..."
1,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...,"[65, 99, 47, 134, 131, 5, 182, 10, 40, 32, 24,..."
2,les là on est très très bon cest bien ouais ce...,516,526,les là on est très très bon cest bien ouais ce...,DZwH8UTZR5E,https://www.youtube.com/watch?v=DZwH8UTZR5E&t=...,"[13, 61, 19, 18, 81, 81, 78, 10, 29, 157, 10, ..."
3,nai pas bien fait parce que cest pas bien de f...,591,596,nai pas bien fait parce que cest pas bien de f...,9RDA9RZLTSc,https://www.youtube.com/watch?v=9RDA9RZLTSc&t=...,"[155, 2, 29, 40, 131, 5, 10, 2, 29, 0, 45, 14,..."
4,et cest la même chose mais cest vrai que cest ...,324,330,et cest la même chose mais cest vrai que cest ...,yQzMFnJEAcw,https://www.youtube.com/watch?v=yQzMFnJEAcw&t=...,"[8, 10, 4, 99, 79, 24, 10, 149, 5, 10, 112, 17..."
...,...,...,...,...,...,...,...
70085,ça va de la,58,60,ça va de la mobilité virtuelle à la mobilité l...,FMKo59medd0,NaN,"[14, 42, 0, 4]"
70086,on la on va,150,151,on la on va essayer de décortiquer commencé à ...,k0CLXlk4z68,NaN,"[19, 4, 19, 42]"
70087,et là ça se,14,15,et là ça se complique,60HvI3CcGFs,NaN,"[8, 61, 14, 48]"
70088,de la de la,685,686,et leurs corps étaient rendus présent par la m...,Wn38oL03yM8,NaN,"[0, 4, 0, 4]"


In [157]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [158]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [159]:
#word_num_index_dict

In [160]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [161]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[133,
 147,
 148,
 149,
 160,
 162,
 170,
 174,
 50,
 181,
 183,
 186,
 194,
 195,
 73,
 80,
 107,
 111,
 116,
 119,
 127]

In [162]:
#Counter(index_list)

In [163]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,comment ça on vous veut du mal pas du tout on ...,261,266,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY,https://www.youtube.com/watch?v=DFejViks2uY&t=...,"[74, 14, 19, 6, 152, 30, 156, 2, 30, 33, 19, 6..."
1,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...,"[65, 99, 47, 134, 131, 5, 182, 10, 40, 32, 24,..."
2,nai pas bien fait parce que cest pas bien de f...,591,596,nai pas bien fait parce que cest pas bien de f...,9RDA9RZLTSc,https://www.youtube.com/watch?v=9RDA9RZLTSc&t=...,"[155, 2, 29, 40, 131, 5, 10, 2, 29, 0, 45, 14,..."
3,dit il sest passé quelque chose il sest passé ...,257,261,dit il sest passé quelque chose il sest passé ...,hecMN45bbK4,https://www.youtube.com/watch?v=hecMN45bbK4&t=...,"[54, 12, 161, 193, 113, 79, 12, 161, 193, 113,..."
4,nom mais je pense que jétais en mieux et peut ...,135,144,nom mais je pense que jétais en mieux et peut ...,c1zhja4N8hw,https://www.youtube.com/watch?v=c1zhja4N8hw&t=...,"[197, 24, 1, 158, 5, 199, 20, 171, 8, 100, 2, ..."
...,...,...,...,...,...,...,...
117,bon où jen suis,320,321,tu as trouvé les verres bon où jen suis daille...,sWVDZ1ZHwJQ,NaN,"[78, 62, 189, 31]"
118,moi quoi bon se,104,107,moi quoi bon se passe là aujourdhui cest un pe...,3NO4xFwNbzA,NaN,"[41, 47, 78, 48]"
119,dis toi tas tas,114,116,dis toi tas tas voté macron toi,XZI0AJ4G7lY,NaN,"[176, 53, 150, 150]"
120,oh oh oh tas,96,98,oh oh oh tas cru que tétais ma meuf,2y9P2YBSuoI,NaN,"[105, 105, 105, 150]"


In [164]:
df_youtube_sent_sample["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_sent_sample['video_id'].map(str)+"&t="+df_youtube_sent_sample['start_time'].map(str)+"s"
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,comment ça on vous veut du mal pas du tout on ...,261,266,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY,https://www.youtube.com/watch?v=DFejViks2uY&t=...,"[74, 14, 19, 6, 152, 30, 156, 2, 30, 33, 19, 6..."
1,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...,"[65, 99, 47, 134, 131, 5, 182, 10, 40, 32, 24,..."
2,nai pas bien fait parce que cest pas bien de f...,591,596,nai pas bien fait parce que cest pas bien de f...,9RDA9RZLTSc,https://www.youtube.com/watch?v=9RDA9RZLTSc&t=...,"[155, 2, 29, 40, 131, 5, 10, 2, 29, 0, 45, 14,..."
3,dit il sest passé quelque chose il sest passé ...,257,261,dit il sest passé quelque chose il sest passé ...,hecMN45bbK4,https://www.youtube.com/watch?v=hecMN45bbK4&t=...,"[54, 12, 161, 193, 113, 79, 12, 161, 193, 113,..."
4,nom mais je pense que jétais en mieux et peut ...,135,144,nom mais je pense que jétais en mieux et peut ...,c1zhja4N8hw,https://www.youtube.com/watch?v=c1zhja4N8hw&t=...,"[197, 24, 1, 158, 5, 199, 20, 171, 8, 100, 2, ..."
...,...,...,...,...,...,...,...
117,bon où jen suis,320,321,tu as trouvé les verres bon où jen suis daille...,sWVDZ1ZHwJQ,https://www.youtube.com/watch?v=sWVDZ1ZHwJQ&t=...,"[78, 62, 189, 31]"
118,moi quoi bon se,104,107,moi quoi bon se passe là aujourdhui cest un pe...,3NO4xFwNbzA,https://www.youtube.com/watch?v=3NO4xFwNbzA&t=...,"[41, 47, 78, 48]"
119,dis toi tas tas,114,116,dis toi tas tas voté macron toi,XZI0AJ4G7lY,https://www.youtube.com/watch?v=XZI0AJ4G7lY&t=...,"[176, 53, 150, 150]"
120,oh oh oh tas,96,98,oh oh oh tas cru que tétais ma meuf,2y9P2YBSuoI,https://www.youtube.com/watch?v=2y9P2YBSuoI&t=96s,"[105, 105, 105, 150]"


In [165]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)

##### Selected Result Word Count

In [166]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,comment ça on vous veut du mal pas du tout on ...,261,266,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY,https://www.youtube.com/watch?v=DFejViks2uY&t=...,"[74, 14, 19, 6, 152, 30, 156, 2, 30, 33, 19, 6..."
1,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...,"[65, 99, 47, 134, 131, 5, 182, 10, 40, 32, 24,..."
2,nai pas bien fait parce que cest pas bien de f...,591,596,nai pas bien fait parce que cest pas bien de f...,9RDA9RZLTSc,https://www.youtube.com/watch?v=9RDA9RZLTSc&t=...,"[155, 2, 29, 40, 131, 5, 10, 2, 29, 0, 45, 14,..."
3,dit il sest passé quelque chose il sest passé ...,257,261,dit il sest passé quelque chose il sest passé ...,hecMN45bbK4,https://www.youtube.com/watch?v=hecMN45bbK4&t=...,"[54, 12, 161, 193, 113, 79, 12, 161, 193, 113,..."
4,nom mais je pense que jétais en mieux et peut ...,135,144,nom mais je pense que jétais en mieux et peut ...,c1zhja4N8hw,https://www.youtube.com/watch?v=c1zhja4N8hw&t=...,"[197, 24, 1, 158, 5, 199, 20, 171, 8, 100, 2, ..."
...,...,...,...,...,...,...,...
117,bon où jen suis,320,321,tu as trouvé les verres bon où jen suis daille...,sWVDZ1ZHwJQ,https://www.youtube.com/watch?v=sWVDZ1ZHwJQ&t=...,"[78, 62, 189, 31]"
118,moi quoi bon se,104,107,moi quoi bon se passe là aujourdhui cest un pe...,3NO4xFwNbzA,https://www.youtube.com/watch?v=3NO4xFwNbzA&t=...,"[41, 47, 78, 48]"
119,dis toi tas tas,114,116,dis toi tas tas voté macron toi,XZI0AJ4G7lY,https://www.youtube.com/watch?v=XZI0AJ4G7lY&t=...,"[176, 53, 150, 150]"
120,oh oh oh tas,96,98,oh oh oh tas cru que tétais ma meuf,2y9P2YBSuoI,https://www.youtube.com/watch?v=2y9P2YBSuoI&t=96s,"[105, 105, 105, 150]"


In [167]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [168]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,va,7
1,voilà,7
2,le,7
3,la,7
4,quelle,7
...,...,...
174,avez,1
175,avant,1
176,aux,1
177,autre,1


In [169]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_Word_Count.xlsx", index=False)

##### Not In Sent And Selected Word List

In [170]:
#lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
317922,puisetil,5
317923,shouzheng,5
317924,steinam,5
317925,cochezle,5


In [171]:
#word_start = 0
#word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
195,nos,332905
196,cela,330730
197,nom,329252
198,prendre,327639


In [172]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [173]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['crois',
 'eu',
 'viens',
 'vrai',
 'vos',
 'mort',
 'lai',
 'vas',
 'sais',
 'vois',
 'désolé',
 'homme',
 'soir',
 'nos',
 'vais',
 'merci',
 'fais',
 'encore',
 'mes',
 'dois',
 'sûr']

In [174]:
set_not_in_sent_word = set(not_in_sent_word)

In [175]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [176]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'crois',
 'dois',
 'désolé',
 'encore',
 'eu',
 'fais',
 'homme',
 'lai',
 'merci',
 'mes',
 'mort',
 'nos',
 'sais',
 'soir',
 'sûr',
 'vais',
 'vas',
 'viens',
 'vois',
 'vos',
 'vrai'}

In [177]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['nos',
 'vais',
 'homme',
 'dois',
 'sûr',
 'sais',
 'vois',
 'vas',
 'vrai',
 'mes',
 'fais',
 'crois',
 'vos',
 'soir',
 'encore',
 'eu',
 'merci',
 'viens',
 'mort',
 'désolé',
 'lai']

In [178]:
len(not_in_sent_selected_word_list)

21

In [179]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,comment ça on vous veut du mal pas du tout on ...,261,266,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY,https://www.youtube.com/watch?v=DFejViks2uY&t=...
1,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...
2,les là on est très très bon cest bien ouais ce...,516,526,les là on est très très bon cest bien ouais ce...,DZwH8UTZR5E,https://www.youtube.com/watch?v=DZwH8UTZR5E&t=...
3,nai pas bien fait parce que cest pas bien de f...,591,596,nai pas bien fait parce que cest pas bien de f...,9RDA9RZLTSc,https://www.youtube.com/watch?v=9RDA9RZLTSc&t=...
4,et cest la même chose mais cest vrai que cest ...,324,330,et cest la même chose mais cest vrai que cest ...,yQzMFnJEAcw,https://www.youtube.com/watch?v=yQzMFnJEAcw&t=...
...,...,...,...,...,...,...
70085,ça va de la,58,60,ça va de la mobilité virtuelle à la mobilité l...,FMKo59medd0,NaN
70086,on la on va,150,151,on la on va essayer de décortiquer commencé à ...,k0CLXlk4z68,NaN
70087,et là ça se,14,15,et là ça se complique,60HvI3CcGFs,NaN
70088,de la de la,685,686,et leurs corps étaient rendus présent par la m...,Wn38oL03yM8,NaN


In [180]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [181]:
if youtube_sent_usage == "all": 
    df_youtube_usage_select = df_youtube_sent_result
else:
    df_youtube_usage_select = df_youtube_sent_select

In [182]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_selected_word_list, df_youtube_usage_select, "search_string", 5, simple=False)
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,nos,estce que nous aussi dans notre vie dans nos,222,226,estce que nous aussi dans notre vie dans nos f...,N2nslH9Ewpw,https://www.youtube.com/watch?v=N2nslH9Ewpw&t=...
1,nos,et cest ce quil nous faut nous aussi en nos,659,662,pour la paix et cest ce quil nous faut nous au...,f2So5lo2riM,https://www.youtube.com/watch?v=f2So5lo2riM&t=...
2,nos,donc là on va prendre le nom de nos,99,103,donc là on va prendre le nom de nos individus ...,Vt7XirF2UA0,https://www.youtube.com/watch?v=Vt7XirF2UA0&t=99s
3,nos,ça on na plus jamais fait avec nos,521,524,ça on na plus jamais fait avec nos enfants,cAMIdz4USkI,https://www.youtube.com/watch?v=cAMIdz4USkI&t=...
4,nos,je viens donc on va parler de nos,347,351,on va commencer par le sud le sud cest doù je ...,tyZSEQdKDcw,https://www.youtube.com/watch?v=tyZSEQdKDcw&t=...
...,...,...,...,...,...,...,...
100,lai,alors quelle est une chose qui je ne lai jamai...,26,30,je suis un londonien alors quelle est une chos...,QiNWTHx9gCQ,https://www.youtube.com/watch?v=QiNWTHx9gCQ&t=26s
101,lai,en vrai je peux pas vraiment parler parce que ...,47,50,pas possible à lire en vrai je peux pas vraime...,SYsh6tP7TZo,https://www.youtube.com/watch?v=SYsh6tP7TZo&t=47s
102,lai,moi je lai donc je peux vraiment vraiment vous...,654,658,moi je lai donc je peux vraiment vraiment vous...,PdDyuT17ikQ,https://www.youtube.com/watch?v=PdDyuT17ikQ&t=...
103,lai,oui je ne lai jamais vu avant mais cest aussi ...,384,387,oui je ne lai jamais vu avant mais cest aussi ...,29VkjY5KO14,https://www.youtube.com/watch?v=29VkjY5KO14&t=...


In [183]:
#not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
#not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
#not_in_sent_selected_word_list_merge

In [184]:
not_in_sent_selected_word_list_result.word.nunique()

21

In [185]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_result["word"].to_list())

In [186]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

set()

In [187]:
#word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

In [188]:
not_in_sent_selected_word_list_result["video_url"] = "https://www.youtube.com/watch?v="+not_in_sent_selected_word_list_result['video_id'].map(str)+"&t="+not_in_sent_selected_word_list_result['start_time'].map(str)+"s"
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,nos,estce que nous aussi dans notre vie dans nos,222,226,estce que nous aussi dans notre vie dans nos f...,N2nslH9Ewpw,https://www.youtube.com/watch?v=N2nslH9Ewpw&t=...
1,nos,et cest ce quil nous faut nous aussi en nos,659,662,pour la paix et cest ce quil nous faut nous au...,f2So5lo2riM,https://www.youtube.com/watch?v=f2So5lo2riM&t=...
2,nos,donc là on va prendre le nom de nos,99,103,donc là on va prendre le nom de nos individus ...,Vt7XirF2UA0,https://www.youtube.com/watch?v=Vt7XirF2UA0&t=99s
3,nos,ça on na plus jamais fait avec nos,521,524,ça on na plus jamais fait avec nos enfants,cAMIdz4USkI,https://www.youtube.com/watch?v=cAMIdz4USkI&t=...
4,nos,je viens donc on va parler de nos,347,351,on va commencer par le sud le sud cest doù je ...,tyZSEQdKDcw,https://www.youtube.com/watch?v=tyZSEQdKDcw&t=...
...,...,...,...,...,...,...,...
100,lai,alors quelle est une chose qui je ne lai jamai...,26,30,je suis un londonien alors quelle est une chos...,QiNWTHx9gCQ,https://www.youtube.com/watch?v=QiNWTHx9gCQ&t=26s
101,lai,en vrai je peux pas vraiment parler parce que ...,47,50,pas possible à lire en vrai je peux pas vraime...,SYsh6tP7TZo,https://www.youtube.com/watch?v=SYsh6tP7TZo&t=47s
102,lai,moi je lai donc je peux vraiment vraiment vous...,654,658,moi je lai donc je peux vraiment vraiment vous...,PdDyuT17ikQ,https://www.youtube.com/watch?v=PdDyuT17ikQ&t=...
103,lai,oui je ne lai jamais vu avant mais cest aussi ...,384,387,oui je ne lai jamais vu avant mais cest aussi ...,29VkjY5KO14,https://www.youtube.com/watch?v=29VkjY5KO14&t=...


In [189]:
not_in_sent_selected_word_list_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [190]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['French_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx',
 'French_200_Word_Group_In_Youtube_Sentence_Word_Index_Word_Count.xlsx',
 'French_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx']

In [191]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [192]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass